In [1]:
pip install imblearn


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from imblearn.combine import SMOTEENN

In [3]:
df=pd.read_csv('tel_churn.csv')
df.head(1)

,Unnamed: 0,SeniorCitizen,MonthlyCharges,TotalCharges,Churn,gender_Male,Partner_Yes,Dependents_Yes,PhoneService_Yes,MultipleLines_No phone service,...,Contract_Two year,PaperlessBilling_Yes,PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check,tenure_group_13 - 24,tenure_group_25 - 36,tenure_group_37 - 48,tenure_group_49 - 60,tenure_group_61 - 72
0,0,0,29.85,29.85,0,0,1,0,0,1,...,0,1,0,1,0,0,0,0,0,0


In [4]:
df=df.drop('Unnamed: 0',axis=1)

In [5]:
X=df.drop('Churn',axis=1)
X

,SeniorCitizen,MonthlyCharges,TotalCharges,gender_Male,Partner_Yes,Dependents_Yes,PhoneService_Yes,MultipleLines_No phone service,MultipleLines_Yes,InternetService_Fiber optic,...,Contract_Two year,PaperlessBilling_Yes,PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check,tenure_group_13 - 24,tenure_group_25 - 36,tenure_group_37 - 48,tenure_group_49 - 60,tenure_group_61 - 72
0,0,29.85,29.85,0,1,0,0,1,0,0,...,0,1,0,1,0,0,0,0,0,0
1,0,56.95,1889.50,1,0,0,1,0,0,0,...,0,0,0,0,1,0,1,0,0,0
2,0,53.85,108.15,1,0,0,1,0,0,0,...,0,1,0,0,1,0,0,0,0,0
3,0,42.30,1840.75,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
4,0,70.70,151.65,0,0,0,1,0,0,1,...,0,1,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7027,0,84.80,1990.50,1,1,1,1,0,1,0,...,0,1,0,0,1,1,0,0,0,0
7028,0,103.20,7362.90,0,1,1,1,0,1,1,...,0,1,1,0,0,0,0,0,0,1
7029,0,29.60,346.45,0,1,1,0,1,0,0,...,0,1,0,1,0,0,0,0,0,0
7030,1,74.40,306.60,1,1,0,1,0,1,1,...,0,1,0,0,1,0,0,0,0,0


In [6]:
y=df['Churn']
y

0       0
1       0
2       1
3       0
4       1
       ..
7027    0
7028    0
7029    0
7030    1
7031    0
Name: Churn, Length: 7032, dtype: int64

#### Train Test Split

In [7]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2)

#### Decision Tree Classifier

In [8]:
model_dt=DecisionTreeClassifier(criterion="gini",random_state=100,max_depth=6,min_samples_leaf=8)

In [9]:
model_dt.fit(X_train,y_train)

DecisionTreeClassifier(max_depth=6, min_samples_leaf=8, random_state=100)

In [10]:
y_pred=model_dt.predict(X_test)
y_pred

array([0, 0, 1, ..., 0, 0, 1], dtype=int64)

In [11]:
model_dt.score(X_test,y_test)

0.7924662402274343

In [12]:
print(classification_report(y_test,y_pred,labels=[0,1]))

              precision    recall  f1-score   support

           0       0.84      0.90      0.87      1050
           1       0.62      0.48      0.54       357

    accuracy                           0.79      1407
   macro avg       0.73      0.69      0.70      1407
weighted avg       0.78      0.79      0.78      1407



In [13]:
sm = SMOTEENN()
X_resampled,y_resampled = sm.fit_resample(X,y)

In [14]:
Xr_train,Xr_test,yr_train,yr_test=train_test_split(X_resampled, y_resampled,test_size=0.2)

In [15]:
model_dt_smote=DecisionTreeClassifier(criterion = "gini",random_state = 100,max_depth=6, min_samples_leaf=8)

In [16]:
model_dt_smote.fit(Xr_train,yr_train)
yr_predict = model_dt_smote.predict(Xr_test)
model_score_r = model_dt_smote.score(Xr_test, yr_test)
print(model_score_r)
print(metrics.classification_report(yr_test, yr_predict))

0.9206896551724137
              precision    recall  f1-score   support

           0       0.95      0.88      0.91       549
           1       0.90      0.96      0.93       611

    accuracy                           0.92      1160
   macro avg       0.92      0.92      0.92      1160
weighted avg       0.92      0.92      0.92      1160



In [17]:
print(metrics.confusion_matrix(yr_test,yr_predict))

[[483  66]
 [ 26 585]]


Accuracy with 92% is a very good recall,precisin and F1 score for minority class

### Random Forest Classifier

In [18]:
from sklearn.ensemble import RandomForestClassifier

In [19]:
model_rf=RandomForestClassifier(n_estimators=100,criterion='gini',random_state=100,max_depth=6, min_samples_leaf=8)

In [20]:
model_rf.fit(X_train,y_train)

RandomForestClassifier(max_depth=6, min_samples_leaf=8, random_state=100)

In [21]:
y_pred=model_rf.predict(X_test)

In [22]:
model_rf.score(X_test,y_test)

0.7910447761194029

In [23]:

print(classification_report(y_test,y_pred,labels=[0,1]))

              precision    recall  f1-score   support

           0       0.81      0.93      0.87      1050
           1       0.65      0.38      0.48       357

    accuracy                           0.79      1407
   macro avg       0.73      0.65      0.67      1407
weighted avg       0.77      0.79      0.77      1407



In [24]:
sm=SMOTEENN()
X_resampled1,y_resampled1 = sm.fit_resample(X,y)

In [25]:
Xr_train1,Xr_test1,yr_train1,yr_test1=train_test_split(X_resampled1,y_resampled1,test_size=0.2)

In [26]:
model_rf_smote=RandomForestClassifier(n_estimators=100,criterion='gini',random_state=100,max_depth=6, min_samples_leaf=8)

In [27]:
model_rf_smote.fit(Xr_train1,yr_train1)

RandomForestClassifier(max_depth=6, min_samples_leaf=8, random_state=100)

In [28]:
yr_predict1=model_rf_smote.predict(Xr_test1)

In [29]:
model_score_r1=model_rf_smote.score(Xr_test1,yr_test1)

In [30]:
print(model_score_r1)
print(metrics.classification_report(yr_test1,yr_predict1))

0.9280070237050044
              precision    recall  f1-score   support

           0       0.94      0.90      0.92       506
           1       0.92      0.95      0.94       633

    accuracy                           0.93      1139
   macro avg       0.93      0.92      0.93      1139
weighted avg       0.93      0.93      0.93      1139



In [31]:
print(metrics.confusion_matrix(yr_test1,yr_predict1))

[[454  52]
 [ 30 603]]


With RF Classifier,also we are able to get quiet good results infact better than decision tree.

### Performing PCA 

In [32]:
#Applying PCA
from sklearn.decomposition import PCA
pca = PCA(0.9)
Xr_train_pca = pca.fit_transform(Xr_train1)
Xr_test_pca = pca.transform(Xr_test1)
explained_variance = pca.explained_variance_ratio_

In [33]:
model=RandomForestClassifier(n_estimators=100,criterion='gini',random_state=100,max_depth=6,min_samples_leaf=8)

In [34]:
model.fit(Xr_train_pca,yr_train1)

RandomForestClassifier(max_depth=6, min_samples_leaf=8, random_state=100)

In [35]:
yr_predict_pca = model.predict(Xr_test_pca)

In [36]:
model_score_r_pca = model.score(Xr_test_pca,yr_test1)

In [37]:
print(model_score_r_pca)
print(metrics.classification_report(yr_test1,yr_predict_pca))

0.7129060579455663
              precision    recall  f1-score   support

           0       0.69      0.65      0.67       506
           1       0.73      0.77      0.75       633

    accuracy                           0.71      1139
   macro avg       0.71      0.71      0.71      1139
weighted avg       0.71      0.71      0.71      1139



With PCA, we couldn't see any better results, hence let's finalise the model which was created by RF

### Pickling the model

In [38]:
import pickle

In [39]:
filename = 'first_telc.csv'

In [40]:
pickle.dump(model_rf_smote,open(filename,'wb'))

In [41]:
load_model = pickle.load(open(filename,'rb'))

In [42]:
model_score_r1 = load_model.score(Xr_test1,yr_test1)

In [43]:
model_score_r1

0.9280070237050044

Our final model i.e. RF Classifier with SMOTEENN, is now ready and dumped in model.sav, which we will use and prepare API's so that we can access our model from UI.